In [ ]:
from pathlib import Path
from oura_analysis.loader import OuraDataTimeseries

import plotly.graph_objects as go
import numpy as np

# load data
data_path = Path("../data") / "sleep_HRV_HR_2019-06-01_2024-01-01.json"

oura_data = OuraDataTimeseries.from_path(data_path)


def normalise_array(input: np.array, desired_length: int) -> np.array:
    # Create an array of equally spaced values from 0 to 1 with desired_length points
    target_positions = np.linspace(0, 1, desired_length)
    # Create an array of equally spaced values from 0 to 1 with the length of the input array
    original_positions = np.linspace(0, 1, len(input))
    # Interpolate the values in the input array to the desired_length
    result = np.interp(target_positions, original_positions, input)
    return result

In [ ]:
def create_plots(input_data):
    normalised_data_for_plot = []
    normalised_data_arrays = []
    original_lengths_hrv = []
    for date, timeseries in input_data.items():
        original_lengths_hrv.append((date, len(timeseries)))
        normalised_timeseries = normalise_array(timeseries, desired_length=100)
        normalised_data_for_plot.append({"date": date, "x": np.arange(len(normalised_timeseries)), "y": normalised_timeseries})
        normalised_data_arrays.append(normalised_timeseries)

    normalised_data_arrays = np.array(normalised_data_arrays)
    mean_values = np.nanmean(normalised_data_arrays, axis=0)
    std_values = np.nanstd(normalised_data_arrays, axis=0)
    upper_bound = mean_values + std_values
    lower_bound = mean_values - std_values

    # fig = go.Figure()
    # # Add traces
    # for data in normalised_data_for_plot:
    #     fig.add_trace(go.Scatter(x=data["x"], y=data["y"], mode="lines",name=data["date"]))

    # fig.show()

    # Define the x-values (assuming a linear range from 0 to 1)
    x_values = np.linspace(0, 1, len(mean_values))

    # Create a Plotly trace for the mean line
    mean_trace = go.Scatter(x=x_values, y=mean_values, mode="lines", name="Mean")

    shaded_area_trace = go.Scatter(
        x=np.concatenate((x_values, x_values[::-1])),
        y=np.concatenate((upper_bound, lower_bound[::-1])),
        fill="tozerox",
        fillcolor="rgba(0,100,80,0.2)",
        line=dict(color="rgba(255,255,255,0)"),
        name="Mean ± Std",
    )

    # Create the layout for the plot
    layout = go.Layout(title="Mean and Standard Deviation Plot", xaxis=dict(title="X-Axis"), yaxis=dict(title="Y-Axis"), showlegend=True)

    # Create the figure and add the traces
    fig = go.Figure(data=[mean_trace, shaded_area_trace], layout=layout)

    # Display the plot
    fig.show()

In [ ]:
# plot HRV data

create_plots(oura_data.hrv())

In [ ]:
# plot HR data
create_plots(oura_data.heart_rate())

In [ ]:
# normalise data to 100 datapoints


import numpy as np

# Your input arrays with different lengths
arrays = [np.array([1, 2, 3]), np.array([4, 5, 6, 7, 8]), np.array([9, 10, 11, 12, 13, 14, 15])]

# Determine the desired length
desired_length = 10

# Initialize an empty array to store the normalized arrays
normalized_arrays = []

for arr in arrays:
    # Use numpy.interp to interpolate the values to the desired length
    normalized_arr = np.interp(np.linspace(0, 1, desired_length), np.linspace(0, 1, len(arr)), arr)
    normalized_arrays.append(normalized_arr)

# Convert the list of arrays into a NumPy array
normalized_arrays = np.array(normalized_arrays)

print(normalized_arrays)